In [ ]:
import sys

#Due to leftoevererrors in Nathan's python installation, some cleaning up occurs here
#sys.path.append("./code/")
#sys.path.remove('/usr/local/lib/python2.7/site-packages') 


import matplotlib.pyplot as plt
%matplotlib inline
import copy
import datetime
import pickle
import gzip
import os,glob
import time
import numpy as np
import pandas as pd
import operator
import random
from  scipy import stats



from scipy.stats import norm
from sklearn.neighbors import KernelDensity


import itertools
#sys.path


import regex as re
import datetime
import math
import time
import itertools

import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools
import plotly.figure_factory as ff

import plotly
plotly.tools.set_credentials_file(username='juliettapc', api_key='nM6iUdx6dGaOiPXQTwpP')   # go to: https://plot.ly/settings/api#/   for a new key if needed

########## to be able to plot offline (without sending the plots to the plotly server every time)
import plotly.offline as offline
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)
################



from IPython.core.display import display,HTML
display(HTML("<style>.container { width:100% !important; }</style>"))  # to make the notebook use the entire width of the browser


In [ ]:
                                  

####  NEW FILE INCLUDING EARLY CITATIONS OF YOUNG REFERENCES:   ../data/df_reference_cite_plos_no_self-cit_one_ref_per_sect_ONLY_ARTICLES_early_cit.pkl
                                            
#%time df_merged = pickle.load(open('../data/df_reference_cite_plos_merged_simplified_added_more_columns_no_self-cit_one_ref_per_sect_ONLY_ARTICLES.pkl', 'rb'))



%time df_merged = pickle.load(open('../data/df_reference_cite_plos_no_self-cit_one_ref_per_sect_ONLY_ARTICLES_early_cit_and_after_accretion_time.pkl', 'rb'))
print ("done loading pickles", df_merged.shape)




df_merged = df_merged[df_merged['cite_count'] != -1]   # i dont know why, but there are 7 occurrences with value -1



plos_df = df_merged.drop_duplicates(subset=['paper_UT'])
print (plos_df.shape)



df_merged.head()

In [ ]:
 ##### Histogram number of occurrences in each section (all papers)
#### FIGURE 1B




v1_string = 'regex_sect_index'
       

years=[2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017] 



    
string_filtering_x = 'paper_cite_count'   # bins by plos' citations on the x-axis ###      
  
string_references_age = ""   #young"#old"  # young # all   for the selection of what references i include
    
string_isolated_ref = ""   #""   #"  #""   # 0  or 1 (or empty string, to include all ref)
  
string_self_ref =0    #""      # ""   #1   # 0  or 1 (or empty string, to include all ref)




######### plos ONE categories. 
string_code_categ="" #  ojo!!! the codes are strings, not integers. if i want to include multiple subjects:  "1 2 8"
 
#  '0': 'Biology and life sciences'             6,032,537
#  '1': 'Computer and information sciences'     1,207,799
#  '10': 'Social sciences'                      755,899
#  '2': 'Earth sciences'                        533,155
#  '3': 'Ecology and environmental sciences'    624,142
#  '4': 'Engineering and technology'            382,247 
#  '5': 'Medicine and health sciences'          4,535,926   
#  '6': 'People and places'                     691,523
#  '7': 'Physical sciences'                     2,100,827
#  '8': 'Research and analysis methods'         3,871,470
#  '9': 'Science policy'                        43,360 


######### plos journals 
string_journal=""

    # PLOS ONE       6,367,070
    # PLOS GENET      149,923
    # PLO NE TR D     138,289   # (neglected tropical diseases)
    # PLOS PATHOG     109,803
    # PLOS COMPUT      77,924
    # PLOS BIOL        56,754
    # PLOS MED         24,506

 

######### WoS subject categories. 
string_plos_field=""#['D CU BIOLOGY']"

# ['D RO MULTIDISCIPLINARY SCIENCES']                                                                                                       4464540
# ['D CU BIOLOGY']                                                                                                                          1055045
# ['D RO MULTIDISCIPLINARY SCIENCES', 'D CU BIOLOGY']                                                                                        847485
# ['D KM GENETICS & HEREDITY']                                                                                                               149923
# ['D YU TROPICAL MEDICINE', 'D TI PARASITOLOGY']                                                                                            138289
# ['D ZE VIROLOGY', 'D QU MICROBIOLOGY', 'D TI PARASITOLOGY']                                                                                109803
# ['D CO BIOCHEMICAL RESEARCH METHODS', 'D MC MATHEMATICAL & COMPUTATIONAL BIOLOGY']                                                          77687
# ['D CQ BIOCHEMISTRY & MOLECULAR BIOLOGY', 'D CU BIOLOGY']                                                                                   56754
# ['D PY MEDICINE, GENERAL & INTERNAL']                                                                                                       24506
# ['D CO BIOCHEMICAL RESEARCH METHODS', 'D MC MATHEMATICAL & COMPUTATIONAL BIOLOGY', 'D PO MATHEMATICS, INTERDISCIPLINARY APPLICATIONS']        237


    
print ("original size:",df_merged.shape)





    


##### preselection by plos year
print (years)
preselection_df = df_merged[df_merged['plos_pub_year'].isin(years)]  
print ("size of preselection1 (by plos years):",preselection_df.shape)





#### i remove self-citations
if (string_self_ref==0) or  ( string_self_ref == 1 ): 
    preselection_df = preselection_df[preselection_df['self_citation']== string_self_ref ]  
    if string_self_ref ==0:
        string_self_ref = ", no self-cit"
    elif string_self_ref ==1:
        string_self_ref = ", only self-cit"



        
        
######### preselection by isolated or group references:
if (string_isolated_ref==0) or  ( string_isolated_ref == 1 ): 
    preselection_df0 = preselection_df[preselection_df['isolated_citation']== string_isolated_ref ]  
    
    if string_isolated_ref ==0:
        string_isolated_ref = ", group ref"
    elif string_isolated_ref ==1:
        string_isolated_ref = ", isolated ref"
else:    
    preselection_df0 = preselection_df   
    print ("size of preselection1 (by isolated/group ref):",preselection_df0.shape, string_isolated_ref)




    
    
    
    
######### preselection by plos ONE subject category:
if string_code_categ=="": 
    preselection_df111 = preselection_df0
else:    
    if " " not in string_code_categ:  # to include one single category
        preselection_df111 = preselection_df0[preselection_df0['categ_codes'].str.contains(string_code_categ)]        
        string_code_categ = " "+dict_code_categ[string_code_categ]  
        
    else:  # if multiple codes-categories
        list_codes = string_code_categ.split(" ")
        print (list_codes)

        if len(list_codes) >= 2:              
            preselection_df111 = preselection_df0[ preselection_df0['categ_codes'].str.contains('|'.join(list_codes)) ]  # to look for partial matches from a list of strings!!!!!
            
       
        string_code_categ = "" 
        for code in list_codes:
            string_code_categ += "-"+dict_code_categ[code] 
            
            
    print (" size of preselection (by plos ONE subject category):",preselection_df111.shape, string_code_categ)
           
        
        
        
        
        
        

######### preselection by plos journal:
if string_journal=="": 
    preselection_df1 = preselection_df111
else:    
    preselection_df1 = preselection_df111[preselection_df111['plos_j1']== string_journal ]  
print (" size of preselection2 (by plos journal):",preselection_df1.shape, string_journal)







######### preselection by plos field:
if string_plos_field=="": 
    preselection_df2 = preselection_df1
else:    
    preselection_df2 = preselection_df1[preselection_df1['plos_field']== string_plos_field ]  
print (" size of preselection2 (by plos field):",preselection_df2.shape, string_plos_field)




preselection_df3 = preselection_df2
   
  



if v1_string ==  'cite_count'  or       v1_string ==  'log_num_cit_ref'   or v1_string == 'log2_num_cit_ref':
     
      
    
    string_age_selection=''

    ##### preselection only young/old references:        
    if string_references_age == "young":
        time_window = 1
        string_age_selection="only young references from >="+ str((min(years)-time_window))
        preselection_df3 = preselection_df2[preselection_df2['ref_pub_year'] >= (min(years)-time_window) ]   
        print ("  size of preselection3 (only young references):",preselection_df3.shape, string_age_selection)
        
    elif string_references_age == "old":
        time_window = 10
        string_age_selection="only old references from <="+str((min(years)-time_window))
        preselection_df3 = preselection_df2[preselection_df2['ref_pub_year'] <= (min(years)-time_window) ]   
        print ("  size of preselection3 (only young references):",preselection_df3.shape,string_age_selection )
        
    else:
        string_age_selection="young&old"       
        print ("  No preselection by age of references:",preselection_df3.shape )
          
    
    

N_plos=len(preselection_df3.paper_UT.unique())        ## this values are overall, for the title   
N_all=len(preselection_df3)
 

print ("\nTot # records included:",len(preselection_df3),"   # number of plos papers:",len(preselection_df3.paper_UT.unique()), "   # unique ref:", len(preselection_df3.reference_UT.unique()),'\n')

 




















x1 = preselection_df3[v1_string]

print (x1.value_counts())

font_gral=60

font=font_gral#20
font_ticks=font_gral#70
font_axes=font_gral#80


trace1= Histogram(
        x=x1, 
     marker=dict(
        color='#9900ff',
    ),
       # name='high cit',
       histnorm='probability',
    #cumulative=dict(enabled=True)
)


layout = Layout(    

    bargap=0.2,

    xaxis=dict(
        #title= 'Section' , #v1_string.replace("_"," "),   
        range=[-.49, 3.49],
        titlefont=dict(            
            size=font_axes,
            color='black'
        #    color='lightgrey'
        ),  
        tickfont=dict(              
            size=font_ticks+30,
            color='black'
        ),
        #type='log'
        tickvals=[0,1,2,3],
        #ticktext=["Introduction","Methods","Results","Discussion"]
        ticktext=["I","M","R","D"]
    ),
    yaxis=dict(
        title='Fraction of references',
        #type='log',
        titlefont=dict(
            size=font_axes+30,
            color='black'
        #    color='lightgrey'
        ),  
        tickfont=dict(               
            size=font_ticks+30,
            color='black'
        ),
        
    ),                

    margin=go.Margin(
        l=350,
       # r=50,
        b=150,
        #t=100,
       # pad=4
    ),



)       

data = [trace1]

fig = Figure(data=data, layout=layout)

iplot(fig, filename='Fraction_references_each_section_overall.html')


offline.plot(fig, auto_open=True, image = 'png', image_filename="fig1b" ,
             output_type='file', image_width=2000, image_height=1400, filename='/home/staff/julia/at_Northwestern/In_Text_Citations/In-Text-Citations-New/plots/fig1b.html', validate=False)


print ("occurrences:", df_merged.shape, "unique ref_UTs:",len(df_merged.reference_UT.unique()))